In [2]:
import pandas as pd
import json
import os

In [5]:
# get all json files in score/ directory
model = 'databricks-dbrx-instruct'
results_dir = f'score/{model}'
json_files = [f'{results_dir}/{f}' for f in os.listdir(results_dir) if f.endswith('.json')]

In [6]:
json_files

['score/databricks-dbrx-instruct/relevance_score.json',
 'score/databricks-dbrx-instruct/executable_simple_score.json',
 'score/databricks-dbrx-instruct/parallel_multiple_function_score.json',
 'score/databricks-dbrx-instruct/executable_multiple_function_score.json',
 'score/databricks-dbrx-instruct/executable_parallel_multiple_function_score.json',
 'score/databricks-dbrx-instruct/simple_score.json',
 'score/databricks-dbrx-instruct/multiple_function_score.json',
 'score/databricks-dbrx-instruct/java_score.json',
 'score/databricks-dbrx-instruct/executable_parallel_function_score.json',
 'score/databricks-dbrx-instruct/rest_score.json',
 'score/databricks-dbrx-instruct/javascript_score.json',
 'score/databricks-dbrx-instruct/parallel_function_score.json']

In [7]:
results_dict = {'filename': [], 'accuracy': [], 'correct_count': [], 'total_count': []}
for filename in json_files:
    with open(filename, 'r') as f:
        data = [json.loads(line) for line in f.readlines()]
        accuracy_info = data[0]
        results_dict['filename'].append(filename)
        for key in accuracy_info.keys():
            results_dict[key].append(accuracy_info[key])

In [8]:
results_df = pd.DataFrame(results_dict)
results_df['metric'] = results_df['filename'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [9]:
results_df[['metric', 'accuracy', 'correct_count', 'total_count']]

,metric,accuracy,correct_count,total_count
0,relevance_score,0.529167,127,240
1,executable_simple_score,0.610000,61,100
2,parallel_multiple_function_score,0.440000,88,200
3,executable_multiple_function_score,0.540000,27,50
4,executable_parallel_multiple_function_score,0.150000,6,40
5,simple_score,0.765000,306,400
6,multiple_function_score,0.810000,162,200
7,java_score,0.390000,39,100
8,executable_parallel_function_score,0.540000,27,50
9,rest_score,0.714286,50,70


In [10]:
results_df.correct_count.sum() / results_df.total_count.sum()

0.6247058823529412